In [2]:
!pip install sympy
!pip install antlr4-python3-runtime==4.10
!pip install latex2sympy2

  Using cached antlr4_python3_runtime-4.10-py3-none-any.whl
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.7.2
    Uninstalling antlr4-python3-runtime-4.7.2:
      Successfully uninstalled antlr4-python3-runtime-4.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
latex2sympy2 1.9.1 requires antlr4-python3-runtime==4.7.2, but you have antlr4-python3-runtime 4.10 which is incompatible.
  Using cached antlr4_python3_runtime-4.7.2-py3-none-any.whl
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.10
    Uninstalling antlr4-python3-runtime-4.10:
      Successfully uninstalled antlr4-python3-runtime-4.10


In [3]:
from sympy.parsing.latex import parse_latex
from sympy import Symbol, Integer, Pow, Mul, Add, srepr, symbols, sympify, S
from sympy.core.basic import Basic

In [5]:
x, y, z = symbols('x y z')

- adciona o argumento
- se abrir parênteses, adciona um nó na esquerda
- conta a profundidade q ta na árvore, para voltar depois
- da para contar a profundidade com o número de parênteses que abriu e fechou, sem considerar o parênteses de Integer e Symbol

In [10]:
import ast

expr = x**2 + x*y + 5*z**3
pseudo_tree = srepr(expr)

class Node:
    def __init__(self, name):
        self.name = name
        self.children = []

def srepr_to_tree(srepr):
    tree = ast.parse(srepr).body[0].value
    return build_tree(tree)

def build_tree(node):
    if isinstance(node, ast.Name):
        return Node(node.id)
    elif isinstance(node, ast.Call):
        parent = Node(node.func.id)
        for arg in node.args:
            parent.children.append(build_tree(arg))
        return parent
    elif isinstance(node, ast.Constant):
        return Node(repr(node.value))  # Use repr() to get the string representation of the value
    else:
        return Node(str(node))

def print_tree(node, indent=0):
    print("  " * indent + node.name)
    for child in node.children:
        print_tree(child, indent + 1)

tree = srepr_to_tree(pseudo_tree)
print_tree(tree)

Add
  Pow
    Symbol
      'x'
    Integer
      2
  Mul
    Symbol
      'x'
    Symbol
      'y'
  Mul
    Integer
      5
    Pow
      Symbol
        'z'
      Integer
        3


In [11]:
from sympy import symbols, S
from sympy.core.basic import Basic

# Define the symbols
x, y, z = symbols('x y z')

# Define the expression
expr = x**2 + x*y + 5*z**3

# Define a simple Tree node
class TreeNode:
    def __init__(self, value):
        self.value = value  # The function or value the node represents
        self.children = []  # List of child nodes

    def add_child(self, child):
        self.children.append(child)

    def __repr__(self, level=0):
        ret = "\t"*level+repr(self.value)+"\n"
        for child in self.children:
            ret += child.__repr__(level+1)
        return ret

# Function to build a tree from a SymPy expression
def build_tree(expr, parent=None):
    if isinstance(expr, Basic):
        if expr.is_Atom:
            # If the expression is an atom (i.e., cannot be decomposed further), it becomes a leaf node
            node = TreeNode(str(expr))
        else:
            # If the expression is not an atom, it is a function or operation with arguments
            node = TreeNode(str(expr.func))
            print(type(expr.args))
            for arg in expr.args:
                child_node = build_tree(arg, node)
                node.add_child(child_node)
    else:
        # For handling numbers and other simple types not covered by Basic
        node = TreeNode(str(expr))
    return node

# Build the tree from the expression
tree = build_tree(expr)
# print(tree)


<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


In [6]:
from sympy import simplify, srepr, Eq
from latex2sympy2 import latex2sympy, latex2latex
from difflib import SequenceMatcher


def simplify_latex_expression(latex_expr):
    return latex2sympy(latex2latex(latex_expr))

def simplify_sympy_expression(sympy_expr):
    return simplify(sympy_expr.doit().doit())

def simpy_to_tree(sympy_expr):
    return srepr(simplify_sympy_expression(sympy_expr))

def latex_to_tree(latex_expr):
    return srepr(simplify_latex_expression(latex_expr))


In [18]:
# Input string representing the expression in SymPy format
input_str = "Add(Mul(Symbol('b'), Symbol('a')), Pow(Symbol('y'), Integer(2)))"

# Evaluate the string to get the SymPy expression object
expr = sympify(input_str)
print(expr)
# Define a simple Tree node
class TreeNode:
    def __init__(self, value):
        # If the value is a class, extract just the class name (function name)
        if isinstance(value, Basic):
            self.value = value.__class__.__name__
        else:
            self.value = value
        self.children = []  # List of child nodes

    def add_child(self, child):
        self.children.append(child)

    def __repr__(self, level=0):
        ret = "\t"*level+repr(self.value)+"\n"
        for child in self.children:
            ret += child.__repr__(level+1)
        return ret

# Function to build a tree from a SymPy expression
def build_tree(expr, parent=None):
    if isinstance(expr, Basic):
        if expr.is_Atom:
            # If the expression is an atom (i.e., cannot be decomposed further), it becomes a leaf node
            node = TreeNode(str(expr))
        else:
            # If the expression is not an atom, it is a function or operation with arguments
            node = TreeNode(expr.func)  # Pass the func directly
            args = sorted(expr.args, key=lambda x: str(x))
            for arg in args:
                child_node = build_tree(arg, node)
                node.add_child(child_node)
    else:
        # For handling numbers and other simple types not covered by Basic
        node = TreeNode(str(expr))
    return node

# Build the tree from the expression
tree = build_tree(expr)
print(tree)

a*b + y**2
<class 'sympy.core.add.Add'>
	<class 'sympy.core.mul.Mul'>
		'a'
		'b'
	<class 'sympy.core.power.Pow'>
		'2'
		'y'

